# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/__init__.py", line 49, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
                         ^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

In [ ]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [ ]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [ ]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [ ]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [ ]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 40s 2ms/step - loss: 0.6706 - accuracy: 0.5857 - val_loss: 0.6299 - val_accuracy: 0.6816
Epoch 2/10
25000/25000 [==============================] - 43s 2ms/step - loss: 0.5907 - accuracy: 0.7100 - val_loss: 0.5737 - val_accuracy: 0.7118
Epoch 3/10
25000/25000 [==============================] - 43s 2ms/step - loss: 0.5228 - accuracy: 0.7493 - val_loss: 0.5284 - val_accuracy: 0.7355
Epoch 4/10
25000/25000 [==============================] - 42s 2ms/step - loss: 0.4744 - accuracy: 0.7763 - val_loss: 0.5026 - val_accuracy: 0.7533
Epoch 5/10
25000/25000 [==============================] - 43s 2ms/step - loss: 0.4416 - accuracy: 0.7963 - val_loss: 0.4831 - val_accuracy: 0.7652
Epoch 6/10
25000/25000 [==============================] - 42s 2ms/step - loss: 0.4176 - accuracy: 0.8108 - val_loss: 0.4715 - val_accuracy: 0.7726
Epoch 7/10
25000/25000 [==============================] - 41s 2ms/st

In [ ]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 3s 127us/step
Test score: 0.4510296483039856
Test accuracy: 0.7875199913978577


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [ ]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 47s 2ms/step - loss: 0.6337 - accuracy: 0.6565 - val_loss: 0.5743 - val_accuracy: 0.7344
Epoch 2/10
25000/25000 [==============================] - 46s 2ms/step - loss: 0.4866 - accuracy: 0.7780 - val_loss: 0.5044 - val_accuracy: 0.7408
Epoch 3/10
25000/25000 [==============================] - 51s 2ms/step - loss: 0.4099 - accuracy: 0.8179 - val_loss: 0.4731 - val_accuracy: 0.7641
Epoch 4/10
25000/25000 [==============================] - 65s 3ms/step - loss: 0.3701 - accuracy: 0.8379 - val_loss: 0.4171 - val_accuracy: 0.8082
Epoch 5/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.3402 - accuracy: 0.8560 - val_loss: 0.3996 - val_accuracy: 0.8189
Epoch 6/10
25000/25000 [==============================] - 68s 3ms/step - loss: 0.3190 - accuracy: 0.8654 - val_loss: 0.4009 - val_accuracy: 0.8171
Epoch 7/10
25000/25000 [==============================] - 67s 3ms/st

In [ ]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 56s 2ms/step - loss: 0.6718 - accuracy: 0.5425 - val_loss: 0.6276 - val_accuracy: 0.6543
Epoch 2/10
25000/25000 [==============================] - 41s 2ms/step - loss: 0.5967 - accuracy: 0.7342 - val_loss: 0.5971 - val_accuracy: 0.7211
Epoch 3/10
25000/25000 [==============================] - 40s 2ms/step - loss: 0.5605 - accuracy: 0.7888 - val_loss: 0.5658 - val_accuracy: 0.7946
Epoch 4/10
25000/25000 [==============================] - 44s 2ms/step - loss: 0.5377 - accuracy: 0.8089 - val_loss: 0.5595 - val_accuracy: 0.7972
Epoch 5/10
25000/25000 [==============================] - 44s 2ms/step - loss: 0.5206 - accuracy: 0.8184 - val_loss: 0.5430 - val_accuracy: 0.7989
Epoch 6/10
25000/25000 [==============================] - 43s 2ms/step - loss: 0.5059 - accuracy: 0.8263 - val_loss: 0.5306 - val_accuracy: 0.8012
Epoch 7/10
25000/25000 [==============================] - 44s 2ms/st

In [ ]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 41s 2ms/step - loss: 0.4484 - accuracy: 0.8431 - val_loss: 0.4879 - val_accuracy: 0.8017
Epoch 2/10
25000/25000 [==============================] - 22s 885us/step - loss: 0.4384 - accuracy: 0.8447 - val_loss: 0.4796 - val_accuracy: 0.8061
Epoch 3/10
25000/25000 [==============================] - 12s 472us/step - loss: 0.4272 - accuracy: 0.8480 - val_loss: 0.4703 - val_accuracy: 0.8071
Epoch 4/10
25000/25000 [==============================] - 13s 515us/step - loss: 0.4172 - accuracy: 0.8513 - val_loss: 0.4676 - val_accuracy: 0.8094
Epoch 5/10
25000/25000 [==============================] - 12s 472us/step - loss: 0.4085 - accuracy: 0.8535 - val_loss: 0.4590 - val_accuracy: 0.8118
Epoch 6/10
25000/25000 [==============================] - 12s 483us/step - loss: 0.4003 - accuracy: 0.8558 - val_loss: 0.4546 - val_accuracy: 0.8114
Epoch 7/10
25000/25000 [==============================] - 

---
### Machine Learning Foundation (C) 2020 IBM Corporation